# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770878751199                   -0.52    9.0    167ms
  2   -2.772145046661       -2.90       -1.32    1.0    106ms
  3   -2.772170512197       -4.59       -2.52    1.0    137ms
  4   -2.772170684469       -6.76       -3.35    1.0    109ms
  5   -2.772170721103       -7.44       -3.88    2.0    139ms
  6   -2.772170722978       -8.73       -5.01    1.0    115ms
  7   -2.772170723013      -10.45       -5.36    1.0    123ms
  8   -2.772170723015      -11.74       -5.79    2.0    141ms
  9   -2.772170723015      -12.89       -6.26    1.0    121ms
 10   -2.772170723015      -13.32       -7.31    1.0    132ms
 11   -2.772170723015   +  -13.78       -8.01    1.0    125ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770708851020                   -0.53    8.0    171ms
  2   -2.7

1.7735580431169493

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770800307436                   -0.52    9.0    198ms
  2   -2.772062021875       -2.90       -1.32    1.0    121ms
  3   -2.772083056422       -4.68       -2.45    1.0    551ms
  4   -2.772083347159       -6.54       -3.15    1.0    108ms
  5   -2.772083417605       -7.15       -4.28    2.0    131ms
  6   -2.772083417736       -9.88       -4.65    1.0    114ms
  7   -2.772083417810      -10.13       -5.72    1.0    135ms
  8   -2.772083417811      -12.08       -6.85    1.0    118ms
  9   -2.772083417811      -13.59       -7.28    2.0    133ms
 10   -2.772083417811      -14.15       -7.73    1.0    121ms
 11   -2.772083417811      -15.35       -8.57    1.0    125ms

Polarizability via ForwardDiff:       1.7725349705022242
Polarizability via finite difference: 1.7735580431169493
